這次的作業是這樣的，希望你能利用本週教的keras functional API，

設計一個mnist的分類模型，這個模型可以一次訓練十個獨立的全連接層，

每一個獨立的全連接層的目標是學習答案的十維向量裡的其中一個element，

每個全連接層有十個神經元，且都跟隨一個average layer把它的輸出做平均，

最後再把這十個層的輸出連接起來，取softmax之後比較結果。

In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
#keras 
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Permute, Conv1D, Conv2D, Add, Conv2DTranspose 
from keras.optimizers import Adam
from keras.layers import Input, Dense, concatenate, Lambda, Conv2D, Reshape, BatchNormalization, Lambda, Activation 
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from keras.activations import softmax
from keras.datasets import mnist
from keras.utils import plot_model

# tensorflow
import tensorflow as tf

#numpy
import numpy as np

Using TensorFlow backend.


In [3]:
# 把mnist load 進來
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# 把每個像素的值從(0~255)->(0, 1)
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples


In [4]:
# 定義一下Average
# 喔，keepdims這個參數在這裏真是他X的重要
def average_function(inputs):                       # 把運算包成 Layer的方法：def 函數變數(輸入):
    return K.mean(inputs, axis=-1, keepdims=True)   # return 用 keras.backend 的運算結果

#把定義好的Average用Lambda封裝起來變成layer
average_layer = Lambda(average_function)            # 變數名稱 = Lambda(函數)

In [5]:
# Model 
model_input = Input(shape=(28, 28))  # 輸入層
flatten = Flatten()(model_input)

# 作業開始

In [6]:
# 請給我十個分開的Dense層，每個層裡面有十個神經元，且輸入都來自flatten
# activation function請用relu

f_1 = Dense(10, activation='relu')
f_2 = Dense(10, activation='relu')
f_3 = Dense(10, activation='relu')
f_4 = Dense(10, activation='relu')
f_5 = Dense(10, activation='relu')
f_6 = Dense(10, activation='relu')
f_7 = Dense(10, activation='relu')
f_8 = Dense(10, activation='relu')
f_9 = Dense(10, activation='relu')
f_10 = Dense(10, activation='relu')

# 每一個Dense層後面都必須再接一個 average_layer，然後把這十個層整理成一個list
# 舉例： 
# x_1 = Dense(10)
# avg_1 = averge_layer(x_1)
# list.append(...)
# 這不是完整的答案，只是個提示。
# 且答案不侷限於上述寫法，有更簡潔明快的做法
q1 = f_1(flatten)
w1 = f_2(flatten)
e1 = f_3(flatten)
r1 = f_4(flatten)
t1 = f_5(flatten)
y1 = f_6(flatten)
u1 = f_7(flatten)
i1 = f_8(flatten)
o1 = f_9(flatten)
p1 = f_10(flatten)

q=average_layer(q1)
w=average_layer(w1)
e=average_layer(e1)
r=average_layer(r1)
t=average_layer(t1)
y=average_layer(y1)
u=average_layer(u1)
i=average_layer(i1)
o=average_layer(o1)
p=average_layer(p1)


# 請把這十個層用學長教過的concatenate層連接起來（基本上就是把上面的average_list放進去並加上其他參數。）
concatenate_layer = concatenate([q,w,e,r,t,y,u,i,o,p])

# Hint: 這次的作業就只有上面的 pass 跟 None 的部分，
# 當然你也可以大方地使用 python 的 list comprehension 功能，
# 這樣可以少寫幾行。

# 作業結束

In [7]:
#加一個softmax
model_output = Activation("softmax")(concatenate_layer)

model = Model(model_input, model_output)


# 你的model.summary理論上會長這樣
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 784)          0           input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10)           7850        flatten_1[0][0]                  
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 10)           7850        flatten_1[0][0]                  
__________________________________________________________________________________________________
dense_3 (D

In [8]:
# compile & fit
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer="adam",
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/10





60000/60000 [==============================] - 34s 571us/step - loss: 0.4801 - acc: 0.8826 - val_loss: 0.2944 - val_acc: 0.9200
Epoch 2/10
60000/60000 [==============================] - 23s 376us/step - loss: 0.2848 - acc: 0.9209 - val_loss: 0.2581 - val_acc: 0.9260
Epoch 3/10
60000/60000 [==============================] - 31s 517us/step - loss: 0.2550 - acc: 0.9288 - val_loss: 0.2373 - val_acc: 0.9330
Epoch 4/10
60000/60000 [==============================] - 19s 316us/step - loss: 0.2359 - acc: 0.9337 - val_loss: 0.2271 - val_acc: 0.9336
Epoch 5/10
60000/60000 [==============================] - 32s 532us/step - loss: 0.2209 - acc: 0.9379 - val_loss: 0.2165 - val_acc: 0.9391
Epoch 6/10
60000/60000 [==============================] - 27s 458us/step - loss: 0.2085 - acc: 0.9409 - val_loss: 0.2087 - val_acc: 0.9389
Epoch 7/10
6

In [10]:
concatenate_layer

<tf.Tensor 'concatenate_1/concat:0' shape=(?, 10) dtype=float32>